# Welcome to Gleeo time analysis notebook

To get started, copy the file **"Time Tracker data export.csv"** next to this notebook and run the cells.

In [ ]:
%autosave 0
%matplotlib inline
import pandas as pd
import numpy as np
# Set ipython's max row display
pd.set_option('display.max_row', 10)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 10)

In [ ]:
time_entries = pd.read_csv("Time Tracker data export.csv")

In [ ]:
time_entries

# Total time spent on projects

All times are displayed in **hours**.

In [ ]:
project_total = time_entries.groupby('Project').sum()
project_total["Decimal Duration"]

In [ ]:
ax = project_total.plot.pie('Decimal Duration', figsize=(10,10), fontsize=15)
ax.set_title('Total time spent on projects')
ax.set_ylabel('')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Time spent on projects every month

In [ ]:
projects = time_entries['Project'].unique()

In [ ]:
time_entries['Start'] = pd.to_datetime(time_entries['Start'])
time_entries['End'] = pd.to_datetime(time_entries['End'])
time_entries = time_entries.set_index(pd.DatetimeIndex(time_entries['Start']))
start = time_entries.iloc[0, time_entries.columns.get_loc('Start')]
end = time_entries.iloc[-1, time_entries.columns.get_loc('Start')]

In [ ]:
def get_time_spent_per(time_unit):
    plt_time_index = pd.date_range(start, end, freq = time_unit)
    plt_df = pd.DataFrame(0, index=plt_time_index, columns=projects)

    for p in projects:
        plt_df[p] = 0

    # Loop through index i, i+1 of plt_df, use them to slice time_entries, sum and populate
    last_idx = plt_df.index[0]
    for idx, plt_row in plt_df.iloc[1:].iterrows():
        matching_entries = time_entries[last_idx:idx]
        matching_entries = matching_entries.groupby('Project').sum()
        for i, p in enumerate(matching_entries.index):
            time_spent = matching_entries.iloc[i, matching_entries.columns.get_loc('Decimal Duration')]
            plt_df.loc[idx, p] += time_spent
        last_idx = idx
    return plt_df

plt_time_unit_params = {
    "figsize": (15,7),
    "fontsize": 15
}

In [ ]:
df_month = get_time_spent_per("M")
ax = df_month.plot.area(title="Time spent on projects every month", **plt_time_unit_params)
average_hours_per_month = df_month.mean().sum()
print("Average hours/month: {}".format(average_hours_per_month))
ax.axhline(average_hours_per_month, linestyle="--", linewidth=3)

# Time spent on projects every week


In [ ]:
df_week = get_time_spent_per("W")
ax = df_week.plot.area(title="Time spent on projects every week", **plt_time_unit_params)
average_hours_per_week = df_week.mean().sum()
print("Average hours/week: {}".format(average_hours_per_week))
ax.axhline(average_hours_per_week, linestyle="--", linewidth=3)